In [29]:
%reload_ext autoreload
%autoreload 2

%gui qt

import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import napari
from pathlib import Path
import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui

In [3]:
# Immediate next steps for programming work:

# - database development
# - ROICaT index alignment tools


In [16]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [113]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'

# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: CR_Hippocannula6/2022-09-05/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
Computing red cell features... (usually takes 10-20 seconds)
C:\Users\andrew\Documents\localData\CR_Hippocannula6\2022-09-05\701


In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [30]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL028','2023-08-08','701')

\\zaru.cortexlab.net\Subjects\ATL028\2023-08-08\701
C:\Users\andrew\Documents\localData\ATL028\2023-08-08\701


In [16]:
dbpath = fm.codePath() / 'database'
headerColumns = []
with open(dbpath/'databaseConstruction'/'sessionHeaders.txt') as f:
    for x in f: 
        headerColumns.append(x.strip())

headerColumns

['unique identifier',
 'mouseName',
 'date',
 'session ID',
 'unique mouse ID',
 'experiment type',
 'imaging',
 'face camera',
 'vrEnvironments',
 'number of imaging planes',
 'separation of imaging planes',
 'pockels cell percentage',
 'rotation of objective',
 'rotation of head plate holder',
 'suite2p processed',
 'suite2p quality control',
 'vrExperiment registration processed',
 'red cell quality control',
 'session quality control']

In [37]:
def checkSessionFiles(mouseName, fileIdentifier, onlyTrue=True):
    dataPath = fm.localDataPath()
    sessionPaths = glob(str(dataPath / mouseName) + '/*')
    for spath in sessionPaths:
        cdate = Path(spath).name
        sessions = glob(spath+'/*/')
        for s in sessions:
            cses = Path(s).name
            targetFiles = glob(s+fileIdentifier)
            numFiles = len(targetFiles)
            if not(onlyTrue and numFiles==0): 
                print(f"{cdate}    {cses}    numFiles: {numFiles}")
            
checkSessionFiles('ATL012', '*eye.mj2') #'*eye.mj2' / 'suite2p')

2023-01-30    701    numFiles: 1
2023-02-01    701    numFiles: 1
2023-02-02    701    numFiles: 1
2023-02-06    701    numFiles: 1
2023-02-07    701    numFiles: 1
2023-02-08    702    numFiles: 1
2023-02-09    701    numFiles: 1
2023-02-14    703    numFiles: 1
2023-02-15    701    numFiles: 1
2023-02-16    701    numFiles: 1
2023-02-17    701    numFiles: 1
2023-02-20    701    numFiles: 1
2023-02-21    702    numFiles: 1
2023-02-22    701    numFiles: 1
2023-02-27    701    numFiles: 1
2023-02-28    701    numFiles: 1
2023-03-01    701    numFiles: 1
2023-03-03    701    numFiles: 1
2023-07-10    701    numFiles: 1
2023-07-10    702    numFiles: 1
2023-07-11    701    numFiles: 1
2023-07-13    701    numFiles: 1
2023-07-14    701    numFiles: 1
2023-07-17    701    numFiles: 1
2023-07-18    701    numFiles: 1
2023-07-19    701    numFiles: 1
2023-07-19    702    numFiles: 1
2023-07-19    703    numFiles: 1
2023-07-20    701    numFiles: 1
2023-07-20    702    numFiles: 1


In [39]:
# Note to self -- I'm currently working on migrating my documentation of the data collection over to the access database. 
# I probably want to restart a new "sessions" database and then move everything over from excel. 
# Right now, I'm just moving data from google sheets to excel...

In [15]:
from glob import glob
dataPath = fm.localDataPath()
glob(str(dataPath / 'ATL012')+'/*/*')

['C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-12\\703',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-18\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-19\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-20\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-20\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-23\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\702',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-24\\703',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-25\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-26\\701',
 'C:\\Users\\andrew\\Documents\\localData\\ATL012\\2023-01-26\\702',
 'C:\\Users\\andrew\\Documents\\lo

In [1]:
%reload_ext autoreload
%autoreload 2

import dbManagement as dbm
vrdb = dbm.vrDatabase()

In [9]:
df = vrdb.showTable()

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [21]:
vrdb.checkS2P(withDatabaseUpdate=True)

Database said suite2p didn't run, but it already did: ATL027/2023-08-08/701
Database said suite2p didn't run, but it already did: ATL027/2023-08-09/701
Database said suite2p didn't run, but it already did: ATL028/2023-08-02/701
Database said suite2p didn't run, but it already did: ATL028/2023-08-03/701
Database said suite2p didn't run, but it already did: ATL028/2023-08-04/701
Database said suite2p didn't run, but it already did: ATL028/2023-08-07/701
Database said suite2p didn't run, but it already did: ATL028/2023-08-08/701
Would try to turn uid 85 row to true with: UPDATE sessiondb SET suite2p = True WHERE [Unique Session ID] = 85;
Would try to turn uid 86 row to true with: UPDATE sessiondb SET suite2p = True WHERE [Unique Session ID] = 86;
Would try to turn uid 197 row to true with: UPDATE sessiondb SET suite2p = True WHERE [Unique Session ID] = 197;
Would try to turn uid 198 row to true with: UPDATE sessiondb SET suite2p = True WHERE [Unique Session ID] = 198;
Would try to turn ui